WE HAVE TO WIN!!

Importing the required dependencies.

In [12]:
# Cell 1: Setup and Library Installation

print("🚀 [1/4] Installing necessary libraries...")
!pip install -q gradio

import os
import tensorflow as tf
from tensorflow.keras import layers, models
import gradio as gr
import numpy as np
import cv2

print("✅ Setup complete. All libraries are ready.")

🚀 [1/4] Installing necessary libraries...
✅ Setup complete. All libraries are ready.


In [13]:
# Cell 2: Model Definition

print("🧠 [2/4] Defining the custom CNN model architecture...")

def get_model(num_classes=4, img_size=128):
    """
    Defines and returns a custom CNN model built from scratch.
    """
    input_shape = (img_size, img_size, 3)

    # Data augmentation layers improve the model's ability to generalize
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ], name='data_augmentation')

    model = models.Sequential([
        layers.Input(shape=input_shape),
        data_augmentation,
        layers.Rescaling(1./255),

        # Convolutional Base for feature extraction
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        # Classifier Head for making predictions
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Dropout helps prevent overfitting
        layers.Dense(num_classes, activation='softmax')
    ], name='Kidney_Anomaly_CNN')

    return model

# You can run this cell to confirm the model structure
model_instance = get_model()
model_instance.summary()

print("\n✅ Model architecture defined successfully.")

🧠 [2/4] Defining the custom CNN model architecture...


Model: "Kidney_Anomaly_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,288,196 (16.36 MB)

 Trainable params: 4,288,196 (16.36 MB)

 Non-trainable params: 0 (0.00 B)


✅ Model architecture defined successfully.


In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityasinghsengar122/analytica-main-problem")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'analytica-main-problem' dataset.
Path to dataset files: /kaggle/input/analytica-main-problem


In [ ]:
# Cell 3: Model Training

print("🏋️ [3/4] Starting the model training process...")

# --- Configuration ---
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = 4

# IMPORTANT: Using the specific file path you provided
BASE_PATH =  '/kaggle/input/analytica-main-problem'
TRAIN_DIR = os.path.join(BASE_PATH, 'train')
TEST_DIR = os.path.join(BASE_PATH, 'test')

# 1. Load Datasets from the specified directory
print(f"Loading data from: {BASE_PATH}")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='int',
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE)
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels='inferred',
    label_mode='int',
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE)
)

CLASS_NAMES = train_dataset.class_names
print(f"Classes found: {CLASS_NAMES}")

# 2. Configure dataset for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# 3. Get and compile the model
model = get_model(num_classes=NUM_CLASSES, img_size=IMG_SIZE)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# 4. Train the model
model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset
)

# 5. Save the final trained model
MODEL_PATH = 'model.h5'
model.save(MODEL_PATH)
print(f"\n✅ Training complete. Model saved as {MODEL_PATH}")

🏋️ [3/4] Starting the model training process...
Loading data from: /kaggle/input/analytica-main-problem
Found 13088 files belonging to 4 classes.
Found 2516 files belonging to 4 classes.
Classes found: ['cysts', 'normal', 'stones', 'tumors']
Epoch 1/25
286/409 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 0.5793 - loss: 1.0278

In [11]:

# ==============================================================================
# ACCURACY REPORTING SECTION
# ==============================================================================

# --- Print Final Test Accuracy ---
print("\n--- Final Model Evaluation on Test Data ---")
loss, accuracy = model.evaluate(test_dataset)
print(f"Final Test Loss: {loss:.4f}")
print(f"**Final Test Accuracy: {accuracy:.4f}**") # This is the key metric

# --- Plot Training & Validation Accuracy Over Time ---
print("\n--- Visualizing Training History ---")
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



--- Final Model Evaluation on Test Data ---


ValueError: You must call `compile()` before using the model.

In [10]:
# Cell 4: Launch the Gradio Web Application (Corrected for Stability)

print("🚀 [4/4] Launching the Gradio web application...")

import gradio as gr
import numpy as np
import cv2
import tensorflow as tf

# --- Load Model and Define Constants ---
MODEL_PATH = 'model.h5'
# Re-compile is set to False for faster loading, as we only need it for inference.
model = tf.keras.models.load_model(MODEL_PATH, compile=False)
CLASS_NAMES = sorted(os.listdir(TRAIN_DIR))


# --- Grad-CAM Functions (for model interpretability) ---
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.layers[-1].output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    # ==============================================================================
    # THE FIX: Handle cases where the gradient is None
    # ==============================================================================
    if grads is None:
        print("Warning: Gradient is None. Returning a zero heatmap.")
        # Return a black image if unable to compute gradients
        return np.zeros(last_conv_layer_output.shape[1:3])
    # ==============================================================================

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Add an epsilon to prevent division by zero
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def superimpose_gradcam(original_img, heatmap):
    img_uint8 = original_img.astype(np.uint8)
    heatmap = cv2.resize(heatmap, (img_uint8.shape[1], img_uint8.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img_uint8, 0.6, heatmap, 0.4, 0)
    return superimposed_img

# --- Main Prediction Function for the App ---
def predict_and_visualize(input_img):
    # Preprocess the image and make a prediction
    img = tf.image.resize(input_img, (IMG_SIZE, IMG_SIZE))
    img_array = tf.expand_dims(img, 0)
    # Use a direct call to the model for prediction
    predictions = model(img_array, training=False)

    confidence_scores = {CLASS_NAMES[i]: float(predictions[0][i]) for i in range(len(CLASS_NAMES))}

    # Generate and overlay the Grad-CAM heatmap
    last_conv_layer_name = [layer.name for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)][-1]
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    superimposed_image = superimpose_gradcam(input_img, heatmap)

    return confidence_scores, superimposed_image

# --- Create and Launch the Gradio Interface ---
iface = gr.Interface(
    fn=predict_and_visualize,
    inputs=gr.Image(type="numpy", label="Upload Kidney Scan"),
    outputs=[
        gr.Label(num_top_classes=4, label="Diagnosis Confidence"),
        gr.Image(type="numpy", label="Model Explanation (Grad-CAM)")
    ],
    title="🩺 Kidney Anomaly Detection AI",
    description="This tool classifies kidney scans into four categories and provides a heatmap to explain the AI's decision."
)

# Use share=True to get a public, shareable link in Google Colab
iface.launch(share=True, debug=True)

🚀 [4/4] Launching the Gradio web application...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47837a746dcbef54bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://47837a746dcbef54bd.gradio.live
